In [1]:
from src.llm_reviewer.llm_api import make_llm_request, LLMAPIFactory
import os
import sys
from project_path import PROJECT_PATH
sys.path.insert(0, PROJECT_PATH)


service_account_path = PROJECT_PATH + "/creds/google__sa.json"
tracking_sheet_id = "1qBU7Kvuuij2fxbqPxebReKMxWgIBmOIE5Gi4ZuX0j_4"
delivery_sheet_id = "1eUif5I8xhHU8fY0X9v8r2JI9hWPh7Dq_9VXpSIHwww4"


In [19]:
llm_client = LLMAPIFactory().get()
make_llm_request(
    llm_client,
    [{'role': 'system', 'content': "How are you today this find evening?"}],
    'gpt-4-1106-preview',
    temperature= 0.0,
    max_tokens = 4000,
    response_format = None,
    retries = 3,
    seed=42
)

"As an AI, I don't have feelings or experiences, but I'm here and ready to assist you with any questions or tasks you have. How can I help you today?"

In [20]:
from src.llm_reviewer.llm_api import global_usage_manager

global_usage_manager.print_costs()

Total cost for all models: $0.001


In [22]:
global_usage_manager.reset_usage()

In [16]:
global_usage_manager._usage_dict

defaultdict(<function src.llm_reviewer.llm_api.GlobalUsageManager.__init__.<locals>.<lambda>()>,
            {'gpt-4-1106-preview': defaultdict(int,
                         {'completion_tokens': 36,
                          'prompt_tokens': 15,
                          'total_tokens': 51})})

In [11]:
77*0.03/1000+30*0.01/1000

0.00261

In [1]:
from src.llm_reviewer.notebook_parser import predict_role


/home/saya/chario/upstream_character_tasks/.venv/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
predict_role([{'role': 'User', 'content': 'how is my favorite GPT doing today?'}, {'role':  '', 'content': 'I am fine thank you'}, {'role':'User', 'content': 'boring as always'}])

Filling out missing header...


('Assistant', None)

In [23]:

from importlib import reload
from src.llm_reviewer import notebook_reviewer
reload(notebook_reviewer)

/home/saya/chario/upstream_character_tasks/.venv/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


<module 'src.llm_reviewer.notebook_reviewer' from '/home/saya/chario/upstream_character_tasks/src/llm_reviewer/notebook_reviewer.py'>

In [24]:
from src.llm_reviewer.notebook_reviewer import review_notebook
import nbformat

fn = 'src/llm_reviewer/tests/samples/10T__ml_research.ipynb'
notebook = nbformat.read(fn, as_version=4)

review = review_notebook(
    {'nb_parsed_notebook': notebook, 'file_id': fn},
    max_threads=10,
    progress_counter= None,
    verbose = 2,
)

Reviews done: 0, Reviews left after this one: 19
Reviews done: 0, Reviews left after this one: 18
Reviews done: 0, Reviews left after this one: 17
Reviews done: 0, Reviews left after this one: 16
Reviews done: 0, Reviews left after this one: 15
Reviews done: 0, Reviews left after this one: 14
Reviews done: 0, Reviews left after this one: 13
Reviews done: 0, Reviews left after this one: 12
Reviews done: 0, Reviews left after this one: 11
Reviews done: 0, Reviews left after this one: 10
Review for turn_id=3 by reviewer='code_reviewer' is done. 1 / 20 reviews completed.
Reviews done: 1, Reviews left after this one: 9
Review for turn_id=1 by reviewer='code_reviewer' is done. 2 / 20 reviews completed.
Reviews done: 2, Reviews left after this one: 8
Review for turn_id=1 by reviewer='english_reviewer' is done. 3 / 20 reviews completed.
Reviews done: 3, Reviews left after this one: 7
Review for turn_id=3 by reviewer='english_reviewer' is done. 4 / 20 reviews completed.
Reviews done: 4, Reviews

In [30]:
global_usage_manager.print_costs()
global_usage_manager._usage_dict

Total cost for all models: $0.333


defaultdict(<function src.llm_reviewer.llm_api.GlobalUsageManager.__init__.<locals>.<lambda>()>,
            {'gpt-4-1106-preview': defaultdict(int,
                         {'completion_tokens': 2555,
                          'prompt_tokens': 25642,
                          'total_tokens': 28197})})

In [31]:
from src.llm_reviewer.turn_reviewer import load_rubric, load_system_prompt


CODE_PROMPT = load_system_prompt('aggregated_reviews_code_judge')
CODE_RUBRICS = load_rubric('code_reviewer')
LANG_PROMPT = load_system_prompt('aggregated_reviews_english_judge')
LANG_RUBRICS = load_rubric('english_reviewer')


from src.llm_reviewer.llm_api import make_llm_request, LLMAPIFactory

def get_judgment(reviewer, feedback):
    llm_client = LLMAPIFactory().get()
    if reviewer == 'code':
        prompt = CODE_PROMPT
        rubrics = CODE_RUBRICS
    elif reviewer == 'lang':
        prompt = LANG_PROMPT
        rubrics = LANG_RUBRICS
    else:
        raise ValueError("Reviewer type not recognized.")
    
    formatted_prompt = prompt.format(FEEDBACK=feedback, GRADING_RUBRICS=rubrics)
    judgment = make_llm_request(
        llm_client,
        [{'role': 'system', 'content': formatted_prompt}],
        'gpt-4-1106-preview',
        temperature= 0.0,
        max_tokens = 4000,
        response_format = {'type': "json_object"},
        retries = 3,
    )
    return judgment

# Example of running the function
#judgment = get_judgment(CODE_PROMPT, gpt_reviews_df.iloc[0]['code_feedback'])
#judgment


In [32]:
from src.llm_reviewer.notebook_reviewer import notebook_reviews_to_df, IssueLevel

df = notebook_reviews_to_df(filter(None, [review]), IssueLevel.MEDIUM)

In [33]:
df

,nb_path,code_score,lang_score,comb_feedback,code_feedback,lang_feedback
0,src/llm_reviewer/tests/samples/10T__ml_researc...,3.5,4.0,#Turn 1:\n\n## Language(4/5):\nNone\n\n## Code...,#Turn 1:\n\n## Code(4/5):\n**Medium_Issues**\n...,#Turn 1:\n\n## Language(4/5):\nNone\n\n======\...


In [34]:
get_judgment('code', df.iloc[0]['code_feedback'])

{'judgment': 'The code provided across turns generally functions correctly but often lacks efficiency, error handling, and proper validation, leading to critical issues that need addressing.',
 'score': '3'}

In [35]:
global_usage_manager.print_costs()
global_usage_manager._usage_dict

Total cost for all models: $0.345


defaultdict(<function src.llm_reviewer.llm_api.GlobalUsageManager.__init__.<locals>.<lambda>()>,
            {'gpt-4-1106-preview': defaultdict(int,
                         {'completion_tokens': 2594,
                          'prompt_tokens': 26731,
                          'total_tokens': 29325})})

In [36]:
0.345/10

0.034499999999999996

In [42]:
0.345*100

34.5

In [38]:
0.333/10

0.0333

In [44]:
(0.0333*4+0.012)*100

14.520000000000003

In [40]:
 0.012

0.012

In [39]:
global_usage_manager.print_costs()
global_usage_manager._usage_dict

Total cost for all models: $0.345


defaultdict(<function src.llm_reviewer.llm_api.GlobalUsageManager.__init__.<locals>.<lambda>()>,
            {'gpt-4-1106-preview': defaultdict(int,
                         {'completion_tokens': 2594,
                          'prompt_tokens': 26731,
                          'total_tokens': 29325})})